In [1]:
import os
import pandas as pd
import numpy as np
import re 

from mlxtend.regressor import StackingRegressor

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
os.getcwd()

'c:\\Users\\dscshap3808\\Documents\\data\\dlwisdom\\mlb'

In [2]:
mlb_in = pd.read_csv('./mlb_in.csv')
mlb_in.dropna(subset=['SKU', 'name'], inplace=True)
(mlb_in).shape


(82387, 5)

In [3]:
mlb_in['top'] = mlb_in['name'].str.extract(r'(衣|夹克|衫|服|马甲|恤)').isnull() != True
mlb_in['top'] = mlb_in['top'].astype(int)
mlb_in['sp'] = mlb_in['name'].str.extract(r'(饰品|配饰)').isnull() != True
mlb_in['sp'] = mlb_in['sp'].astype(int)
mlb_in['cap'] = mlb_in['name'].str.extract(r'(.+CAP|帽)').isnull() != True
mlb_in['cap'] = mlb_in['cap'].astype(int)
mlb_in['pack'] = mlb_in['name'].str.extract(r'(.+PACK|包)').isnull() != True
mlb_in['pack'] = mlb_in['pack'].astype(int)

mlb_in['female'] = mlb_in['name'].str.extract(r'(女士)').isnull() != True
mlb_in['female'] = mlb_in['female'].astype(int)
mlb_in['pants'] = mlb_in['name'].str.extract(r'(裤|PANT)').isnull() != True
mlb_in['pants'] = mlb_in['pants'].astype(int)
mlb_in['OVERSIZED'] = mlb_in['name'].str.extract(r'(.+OVERSIZED)').isnull() != True
mlb_in['OVERSIZED'] = mlb_in['OVERSIZED'].astype(int)
 
mlb_in['DAIMAROO'] = mlb_in['name'].str.extract(r'(DAIMAROO)').isnull() != True
mlb_in['DAIMAROO'] = mlb_in['DAIMAROO'].astype(int)
mlb_in['COMFORTABLE'] = mlb_in['name'].str.extract(r'(.+COMFORTABLE)').isnull() != True
mlb_in['COMFORTABLE'] = mlb_in['COMFORTABLE'].astype(int)

mlb_in['tmjl'] = mlb_in['name'].str.extract(r'(天猫精灵)').isnull() != True
mlb_in['tmjl'] = mlb_in['tmjl'].astype(int)
mlb_in['share'] = mlb_in['name'].str.extract(r'(共用)').isnull() != True
mlb_in['share'] = mlb_in['share'].astype(int)
mlb_in['shoe'] = mlb_in['name'].str.extract(r'(鞋)').isnull() != True
mlb_in['shoe'] = mlb_in['shoe'].astype(int)
mlb_in['cup'] = mlb_in['name'].str.extract(r'(杯)').isnull() != True
mlb_in['cup'] = mlb_in['cup'].astype(int)
mlb_in['textile'] = mlb_in['name'].str.extract(r'(纺织品)').isnull() != True
mlb_in['textile'] = mlb_in['textile'].astype(int)

mlb_in['socks'] = mlb_in['name'].str.extract(r'(袜)').isnull() != True
mlb_in['socks'] = mlb_in['socks'].astype(int)

mlb_in['dress'] = mlb_in['name'].str.extract(r'(裙)').isnull() != True
mlb_in['dress'] = mlb_in['dress'].astype(int)
mlb_in['suit'] = mlb_in['name'].str.extract(r'(套装)').isnull() != True
mlb_in['suit'] = mlb_in['suit'].astype(int)
mlb_in['fan'] = mlb_in['name'].str.extract(r'(风扇)').isnull() != True
mlb_in['fan'] = mlb_in['fan'].astype(int)




In [4]:
mlb_in['else'] = 0
mlb_in.loc[mlb_in.iloc[:,5:].sum(axis = 1) == 0, 'else'] = 1
# mlb_in[mlb_in.iloc[:,5:].sum(axis = 1)  == 0]


In [5]:
mlb_in['year'] = mlb_in['time'].str.extract(r'(.{4})')
mlb_in['month']= mlb_in['time'].str.extract(r'(?<=/)(\d+)')
mlb_in['date'] = mlb_in['time'].str.extract(r'(?<=/\d/)(\d+)')

In [6]:
# mlb_quant = pd.DataFrame(
#     mlb_in.groupby([mlb_in['month'], mlb_in['date'], mlb_in['']]).sum()['quant'])
mlb_in_train = pd.DataFrame(mlb_in.groupby(['year','month','date'], as_index = False).sum('quant'))

cols = ['year','month','date']
mlb_in_train['yearmon'] = mlb_in_train[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
mlb_in_train['yearmon'] = pd.to_datetime(mlb_in_train['yearmon'])



In [7]:
mlb_in_train = mlb_in_train.sort_values(['yearmon'])

In [8]:
# mlb_in_train[mlb_in_train['month'] == "6"]
mlb_in_train.shape[0]/7

24.0

In [9]:
import datetime

In [10]:
# tests # sunday start
wee = list([1,0,0,1,1,1,1])
wee = wee*24
mlb_in_train['wee'] = wee
mlb_in_train.drop('yearmon', axis = 1)

,year,month,date,quant,top,sp,cap,pack,female,pants,...,share,shoe,cup,textile,socks,dress,suit,fan,else,wee
0,2021,1,1,42.0,21,6,3,13,0,0,...,23,2,0,6,0,0,0,0,0,1
11,2021,1,2,11295.0,147,11,7,11,0,18,...,27,0,0,5,0,0,0,0,0,0
22,2021,1,3,21334.0,24,6,5,11,3,26,...,10,49,0,1,0,1,0,0,0,0
24,2021,1,4,39143.0,96,12,43,12,3,5,...,31,2,0,3,0,1,0,0,0,1
25,2021,1,5,9252.0,37,21,11,12,2,1,...,45,2,0,4,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2021,6,17,36348.0,38,4,36,0,0,3,...,6,70,0,0,0,3,0,0,0,0
157,2021,6,18,54433.0,439,7,41,4,0,11,...,10,135,2,0,0,9,0,2,0,1
158,2021,6,19,22149.0,79,7,25,3,1,17,...,13,91,2,0,0,14,0,0,0,1
160,2021,6,20,18994.0,91,12,17,2,2,62,...,25,101,8,0,0,15,0,2,0,1


In [14]:
mlb_in_train_1 = mlb_in_train[mlb_in_train['month'] != "6"]
mlb_in_tests_1 = mlb_in_train[mlb_in_train['month'] == "6"]

In [15]:
mlb_in_train.head()

,year,month,date,quant,top,sp,cap,pack,female,pants,...,shoe,cup,textile,socks,dress,suit,fan,else,yearmon,wee
0,2021,1,1,42.0,21,6,3,13,0,0,...,2,0,6,0,0,0,0,0,2021-01-01,1
11,2021,1,2,11295.0,147,11,7,11,0,18,...,0,0,5,0,0,0,0,0,2021-01-02,0
22,2021,1,3,21334.0,24,6,5,11,3,26,...,49,0,1,0,1,0,0,0,2021-01-03,0
24,2021,1,4,39143.0,96,12,43,12,3,5,...,2,0,3,0,1,0,0,0,2021-01-04,1
25,2021,1,5,9252.0,37,21,11,12,2,1,...,2,0,4,0,0,0,0,0,2021-01-05,1


---

undone part 624

In [16]:
mlb_in.columns[[-2,-1,]]

Index(['month', 'date'], dtype='object')

In [17]:
mlb_in.columns[5:-3]

Index(['top', 'sp', 'cap', 'pack', 'female', 'pants', 'OVERSIZED', 'DAIMAROO',
       'COMFORTABLE', 'tmjl', 'share', 'shoe', 'cup', 'textile', 'socks',
       'dress', 'suit', 'fan', 'else'],
      dtype='object')

In [144]:
# list(mlb_in.columns[5:])
# list(mlb_in.columns[[2,3]])
mlb_long = mlb_in.melt(
    id_vars = list(mlb_in.columns[[2, -2,-1,]]),value_vars=list(mlb_in.columns[5:-3]))

In [145]:
mlb_long['variable'].unique()

array(['top', 'sp', 'cap', 'pack', 'female', 'pants', 'OVERSIZED',
       'DAIMAROO', 'COMFORTABLE', 'tmjl', 'share', 'shoe', 'cup',
       'textile', 'socks', 'dress', 'suit', 'fan', 'else'], dtype=object)

In [194]:
# mlb_long

In [195]:
# mlb_long.drop('value', axis = 1).groupby(['month', 'date','variable']).sum('quant')

In [196]:
# mlb_long.groupby(
    # ['month','date','variable']).sum()

In [197]:
# mlb_fin = pd.DataFrame(mlb_fin)

In [198]:
# mlb_quant .merge(mlb_fin, on=['month', 'date'], how = 'outer')

---
undone part 624 end

code


In [199]:
# (mlb_in['code'].str.extract(r'(^[A-Z])').isnull() != True).sum(axis =1)

## model

In [18]:
# mlb_in_train_1
# mlb_in_tests_1
mlb_in_train_tar = mlb_in_train_1.pop('quant')
mlb_in_train_tar = list(mlb_in_train_tar)
mlb_in_train_1 = mlb_in_train_1.drop('yearmon', axis = 1)
mlb_in_train_1['year'] = mlb_in_train_1['year'].astype(int)
mlb_in_train_1['month'] = mlb_in_train_1['month'].astype(int)
mlb_in_train_1['date'] = mlb_in_train_1['date'].astype(int)

mlb_in_tests_1 = mlb_in_tests_1.drop('yearmon', axis = 1)
mlb_in_tests_1['year'] = mlb_in_tests_1['year'].astype(int)
mlb_in_tests_1['month'] = mlb_in_tests_1['month'].astype(int)
mlb_in_tests_1['date'] = mlb_in_tests_1['date'].astype(int)

In [19]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(n_splits = 5, test_size = .2, train_size = .8, random_state = 529)


In [20]:
alg1 = LinearRegression()
cv_results_1 = model_selection.cross_validate(
    alg1, mlb_in_train_1, mlb_in_train_tar, 
    cv  = cv_split, 
    scoring='neg_mean_absolute_percentage_error')
cv_results_1['test_score'].mean()

-15.700188454108854

In [21]:
alg2 = Ridge()
cv_results_2 = model_selection.cross_validate(alg2, mlb_in_train_1, mlb_in_train_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_2['test_score'].mean()

-15.815162577517702

In [22]:
from sklearn.neural_network import MLPRegressor
alg3 = MLPRegressor(
    hidden_layer_sizes = (345, 31, 1), 
    max_iter=1000,verbose=True, 
    activation = 'relu', 
    solver = 'adam', 
    alpha = .05,
    batch_size = 15,
    # momentum = .9,
    learning_rate = 'adaptive')

cv_results_3 = model_selection.cross_validate(
    alg3, mlb_in_train_1, mlb_in_train_tar, 
    cv  = cv_split, 
    scoring='neg_mean_absolute_percentage_error')
cv_results_3['test_score'].mean()

Iteration 1, loss = 2601719321.04825783
Iteration 2, loss = 2585686840.19755507
Iteration 3, loss = 2569370254.25047970
Iteration 4, loss = 2543894775.93331957
Iteration 5, loss = 2521127443.47089291
Iteration 6, loss = 2477176688.88135338
Iteration 7, loss = 2430969016.95170879
Iteration 8, loss = 2379571789.93371630
Iteration 9, loss = 2308692585.49478006
Iteration 10, loss = 2236161676.80604792
Iteration 11, loss = 2175202850.31364059
Iteration 12, loss = 2093817238.64541221
Iteration 13, loss = 2019310945.11945224
Iteration 14, loss = 1966188709.00198221
Iteration 15, loss = 1922192438.02109504
Iteration 16, loss = 1892338493.98472357
Iteration 17, loss = 1882802641.36081433
Iteration 18, loss = 1870697671.30116653
Iteration 19, loss = 1852236839.33605433
Iteration 20, loss = 1836483654.02431011
Iteration 21, loss = 1826489416.09533477
Iteration 22, loss = 1811080377.91293001
Iteration 23, loss = 1802143336.16510224
Iteration 24, loss = 1798614378.29835439
Iteration 25, loss = 1781

-8.076457032093304

In [23]:
alg3
# cv_results_3['test_score'].mean()
# MLPRegressor(alpha=0.05, batch_size=15, hidden_layer_sizes=(345, 31, 1),
            #  learning_rate='adaptive', max_iter=1000, verbose=True)

MLPRegressor(alpha=0.05, batch_size=15, hidden_layer_sizes=(345, 31, 1),
             learning_rate='adaptive', max_iter=1000, verbose=True)

In [24]:
# alg3.fit()

In [357]:
from sklearn.neural_network import MLPRegressor
alg4 = MLPRegressor(
    hidden_layer_sizes = (345, 31,1), 
    max_iter=1000,verbose=True, 
    activation = 'relu', 
    solver = 'sgd', 
    alpha = .0005,
    batch_size = 100,
    momentum = .001,
    learning_rate_init = .02,
    learning_rate = 'adaptive')

cv_results_4 = model_selection.cross_validate(
    alg4, mlb_in_train_1, mlb_in_train_tar, 
    cv  = cv_split, 
    scoring='neg_mean_absolute_percentage_error')


In [263]:
cv_results_4['test_score'].mean()

-1.3104393372699292e+18

In [254]:
#

---

In [358]:
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(
    n_splits = 6, test_size = .15, train_size = .5, random_state = 529)
alg5 = Lasso()

cv_results_5 = model_selection.cross_validate(
    alg5, mlb_in_train_1, mlb_in_train_tar, 
    cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_5['test_score'].mean()

In [25]:
from sklearn.ensemble import RandomForestRegressor
alg6 = RandomForestRegressor(n_estimators = 11, 
        max_depth = 31,
        min_samples_split = 3, 
        min_samples_leaf=2, 
        # max_leaf_nodes = 31
        )
cv_results_6 = model_selection.cross_validate(
    alg6, mlb_in_train_1, mlb_in_train_tar, 
    cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_6['test_score'].mean()

-17.731608588325205

## test

In [26]:
test_quant_real = mlb_in_tests_1.pop('quant')
test_quant_real.reset_index(drop = True, inplace = True)

In [27]:
fit3 = alg3.fit(mlb_in_train_1, mlb_in_train_tar)

Iteration 1, loss = 2194307475.07488298
Iteration 2, loss = 2194307209.87118387
Iteration 3, loss = 2194306404.63462210
Iteration 4, loss = 2183854368.16920567
Iteration 5, loss = 2150902666.77700090
Iteration 6, loss = 2101298030.92456055
Iteration 7, loss = 2032545357.34073639
Iteration 8, loss = 1915217465.00173259
Iteration 9, loss = 1815867077.67765784
Iteration 10, loss = 1711482478.52121854
Iteration 11, loss = 1637867112.59546423
Iteration 12, loss = 1584852462.14645147
Iteration 13, loss = 1557414623.11439657
Iteration 14, loss = 1548971690.62828970
Iteration 15, loss = 1528977404.32687187
Iteration 16, loss = 1521736913.60599375
Iteration 17, loss = 1509617128.34742188
Iteration 18, loss = 1494019720.19757652
Iteration 19, loss = 1500808302.35036755
Iteration 20, loss = 1481284682.38902020
Iteration 21, loss = 1461445002.12625766
Iteration 22, loss = 1451093041.04295349
Iteration 23, loss = 1441095759.68513680
Iteration 24, loss = 1434952641.66782236
Iteration 25, loss = 1424

In [28]:
fit3_pre = pd.concat(
    [pd.DataFrame(fit3.predict(mlb_in_tests_1)), test_quant_real],
    axis=1,ignore_index = True
);

fit3_pre

,0,1
0,7719.590153,7295.0
1,10711.178210,11790.0
2,13333.246788,27832.0
3,5076.192130,172755.0
4,2.156223,7432.0
5,1466.173465,80613.0
6,19900.305822,10810.0
7,14408.699891,16623.0
8,21276.035070,58582.0
9,32343.864237,13519.0


0       7295.0
1      11790.0
2      27832.0
3     172755.0
4       7432.0
5      80613.0
6      10810.0
7      16623.0
8      58582.0
9      13519.0
10     17608.0
11      4949.0
12     25481.0
13     36348.0
14     54433.0
15     22149.0
Name: 1, dtype: float64

In [36]:
from sklearn import metrics
metrics.mean_absolute_percentage_error(fit3_pre.iloc[:16,1],fit3_pre.iloc[:16,0])

0.747412532781606

In [266]:
fit6 = alg6.fit(mlb_in_train_1, mlb_in_train_tar)


In [295]:
fit6_pre = pd.concat(
    [pd.DataFrame(fit6.predict(mlb_in_tests_1)), test_quant_real],
    axis=1,ignore_index = True
);

fit6_pre

,0,1
0,6749.954113,7295.0
1,9751.604545,11790.0
2,16504.612121,27832.0
3,29236.919697,172755.0
4,21292.241185,7432.0
5,22772.188095,80613.0
6,26195.334848,10810.0
7,24815.368457,16623.0
8,26314.563853,58582.0
9,33401.931818,13519.0


In [364]:
from sklearn import metrics
metrics.mean_absolute_percentage_error(fit6_pre[1],fit6_pre[0])

3.2184286168162735

In [269]:
import time #implementing in this function the time spent on training the model
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

def modelfit(alg, dtrain, target, only_predict = False):
    #Fit the algorithm on the data
    time_start = time.perf_counter() #start counting the time
    if not only_predict:
        alg.fit(dtrain, target)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    
    kfolds = KFold(n_splits=6, shuffle=True, random_state=42)
    
    cv_score = cross_val_score(alg, dtrain,target, cv=kfolds, scoring='neg_mean_squared_error')
    cv_score = np.sqrt(-cv_score)
    
    time_end = time.perf_counter()
    
    total_time = time_end-time_start
    #Print model report:
    print("\nModel Report")
    print("RMSE :  {:.4f}".format(np.sqrt(mean_squared_error(target, dtrain_predictions))))
    print("CV Score : Mean -  %.4f | Std -  %.4f | Min -  %.4f | Max - %.4f" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    print("Amount of time spent during training the model and cross validation: %4.3f seconds" % (total_time))
    
def plot_feature_importance(model, df):
    feature_importance = model.feature_importances_[:30]
    # make importances relative to max importance
    plt.figure(figsize=(20, 20)) #figure size
    feature_importance = 100.0 * (feature_importance / feature_importance.max()) #making it a percentage relative to the max value
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, df.columns[sorted_idx], fontsize=15) #used train_drop here to show the name of each feature instead of our train_prepared 
    plt.xlabel('Relative Importance', fontsize=20)
    plt.ylabel('Features', fontsize=20)
    plt.title('Variable Importance', fontsize=30)

In [271]:
lin_reg = LinearRegression()
modelfit(lin_reg, mlb_in_train_1, mlb_in_train_tar)


Model Report
RMSE :  31750.6865
CV Score : Mean -  61919.0978 | Std -  22175.4367 | Min -  40175.2052 | Max - 105839.9921
Amount of time spent during training the model and cross validation: 0.246 seconds


In [293]:
forest_reg = RandomForestRegressor(n_estimators=3000, 
                                   random_state=529, 
                                   min_samples_leaf=5,
                                   min_samples_split = 5,
                                   max_depth = 25,
                                   n_jobs=-1, oob_score=True)
modelfit(forest_reg, mlb_in_train_1, mlb_in_train_tar)


Model Report
RMSE :  43667.1852
CV Score : Mean -  46620.5247 | Std -  27659.3376 | Min -  18933.1537 | Max - 102307.9280
Amount of time spent during training the model and cross validation: 85.311 seconds


In [277]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=421, max_depth = 15, min_samples_split = 5)
modelfit(tree_reg, mlb_in_train_1, mlb_in_train_tar) 


Model Report
RMSE :  19780.1639
CV Score : Mean -  88137.3955 | Std -  42987.0781 | Min -  26775.9041 | Max - 162303.5689
Amount of time spent during training the model and cross validation: 0.159 seconds


In [278]:
 
from sklearn.ensemble import AdaBoostRegressor

# tree_ada = DecisionTreeRegressor(random_state = 42,max_depth = 4)

ada_reg = AdaBoostRegressor(
    tree_reg, n_estimators=300, random_state=42,learning_rate=0.009, loss='square')
modelfit(ada_reg, mlb_in_train_1, mlb_in_train_tar)

# 181.589


Model Report
RMSE :  1643.2120
CV Score : Mean -  41260.3441 | Std -  21268.5175 | Min -  16776.4402 | Max - 68221.0368
Amount of time spent during training the model and cross validation: 10.713 seconds


In [294]:
forest_reg_pre = pd.concat(
    [pd.DataFrame(forest_reg.predict(mlb_in_tests_1)), test_quant_real], 
    axis = 1,
    join = 'outer',
    ignore_index = True);
forest_reg_pre

,0,1
0,8384.514687,7295.0
1,11471.869326,11790.0
2,16526.981927,27832.0
3,23098.157550,172755.0
4,17963.921048,7432.0
5,26623.122254,80613.0
6,26998.857863,10810.0
7,23036.273727,16623.0
8,27304.401138,58582.0
9,35222.814740,13519.0


In [366]:
metrics.mean_absolute_percentage_error(forest_reg_pre[1],forest_reg_pre[0])

3.5203720550254523

In [292]:
ada_reg_pre = pd.concat(
    [pd.DataFrame(ada_reg.predict(mlb_in_tests_1)), test_quant_real], 
    axis = 1,
    join = 'outer',
    ignore_index = True);
ada_reg_pre

,0,1
0,1495.75,7295.0
1,12704.25,11790.0
2,13367.00,27832.0
3,24983.00,172755.0
4,24126.00,7432.0
5,20542.25,80613.0
6,21303.00,10810.0
7,28242.00,16623.0
8,24965.00,58582.0
9,20873.00,13519.0


In [373]:
metrics.mean_absolute_percentage_error(ada_reg_pre[1],ada_reg_pre[0])

3.7872460788382796

In [378]:
metrics.mean_absolute_error(ada_reg_pre[1],ada_reg_pre[0])

25047.805555555555

In [381]:
sum(abs((ada_reg_pre[1]-ada_reg_pre[0])/ada_reg_pre[1]))/18

3.7872460788382796

lgbm

In [37]:
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_percentage_error

In [39]:
def objective(trial, X=mlb_in_train_1, y=mlb_in_train_tar):

    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=meta_random_seed)


    lgb_params={
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2),
        'max_depth': trial.suggest_int('max_depth', 6, 31),
        'num_leaves': trial.suggest_int('num_leaves', 31, 120),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-1, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-1, 1),
        'random_state': meta_random_seed,
        'metric': 'mse',
        'n_estimators': trial.suggest_int('n_estimators', 6, 3000),
        'n_jobs': -1,
        'cat_feature': [x for x in range(len(categorical_columns))],
        'bagging_seed': 2021,
        'feature_fraction_seed': 2021,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 500),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'max_bin': trial.suggest_int('max_bin', 128, 1024),
        'min_data_per_group': trial.suggest_int('min_data_per_group', 5, 50),
        'cat_smooth': trial.suggest_int('cat_smooth', 10, 250),
        'cat_l2': trial.suggest_int('cat_l2', 1, 20)
    }

    lgb = LGBMRegressor(
        **lgb_params
    )
    lgb.fit(
        X_train,
        y_train,
        eval_set=(X_test,y_test),
        eval_metric='mean_absolute_percentage_error',
        early_stopping_rounds=100,
        verbose=False
    )
    predictions=lgb.predict(X_test)

    return mean_absolute_percentage_error(y_test, predictions)

In [42]:
meta_random_seed = 529
categorical_columns=['']
study = optuna.create_study(direction='minimize')
study.optimize(objective, timeout=3600*7, n_trials=25)

[I 2021-06-24 16:18:06,329] A new study created in memory with name: no-name-35c5762b-c4ea-45db-b74b-c4fd74ea4b8c
[I 2021-06-24 16:18:06,411] Trial 0 finished with value: 77.68637069145414 and parameters: {'learning_rate': 0.007674915293680241, 'max_depth': 7, 'num_leaves': 114, 'reg_alpha': 0.9067163907386172, 'reg_lambda': 0.32198158510100305, 'n_estimators': 998, 'colsample_bytree': 0.7311765774956949, 'min_child_samples': 399, 'subsample_freq': 10, 'subsample': 0.706755721969521, 'max_bin': 345, 'min_data_per_group': 44, 'cat_smooth': 136, 'cat_l2': 19}. Best is trial 0 with value: 77.68637069145414.
[I 2021-06-24 16:18:06,464] Trial 1 finished with value: 77.68637069145414 and parameters: {'learning_rate': 0.0012696940324347491, 'max_depth': 25, 'num_leaves': 104, 'reg_alpha': 0.5235199216434151, 'reg_lambda': 0.7062441793935992, 'n_estimators': 2772, 'colsample_bytree': 0.7990863423157377, 'min_child_samples': 80, 'subsample_freq': 1, 'subsample': 0.7386705421249349, 'max_bin': 5

In [43]:
study.best_params


{'learning_rate': 0.005136975656960963,
 'max_depth': 9,
 'num_leaves': 55,
 'reg_alpha': 0.20817676381652056,
 'reg_lambda': 0.3934488653297847,
 'n_estimators': 255,
 'colsample_bytree': 0.898763145328619,
 'min_child_samples': 32,
 'subsample_freq': 4,
 'subsample': 0.8626632385864561,
 'max_bin': 170,
 'min_data_per_group': 32,
 'cat_smooth': 77,
 'cat_l2': 20}

In [ ]:
mlb_in_train_tar = pd.DataFrame(mlb_in_train_tar)

In [55]:
from sklearn.model_selection import StratifiedKFold,RepeatedKFold
from scipy.special import expit
from sklearn.calibration import CalibratedClassifierCV

random_state = 1026529
n_folds = 6
k_fold = RepeatedKFold(n_splits=n_folds, random_state=random_state)

# y = train_df["target"]

# xgb_train_preds = np.zeros(len(train_df.index), )
# xgb_test_preds = np.zeros(len(test_df.index), )
# xgb_features = xgb_cat_features + list(numerical_columns)

lgbm_train_preds = np.zeros(len(mlb_in_train_1.index), )
lgbm_test_preds = np.zeros(len(mlb_in_train_tar.index), )
# lgbm_features = lgb_cat_features + list(numerical_columns)

# cb_train_preds = np.zeros(len(train_df.index), )
# cb_test_preds = np.zeros(len(test_df.index), )
# cb_features = cb_cat_features + list(numerical_columns)

# ridge_train_preds = np.zeros(len(train_df.index), )
# ridge_test_preds = np.zeros(len(test_df.index), )
# ridge_features = ridge_cat_features + list(numerical_columns)

In [65]:
train_df = mlb_in_train_1
y = mlb_in_train_tar
test_df = mlb_in_tests_1

In [72]:
for fold, (train_index, test_index) in enumerate(
    k_fold.split(train_df, y)):
    print("--> Fold {}".format(fold + 1))
    y_train = y.iloc[train_index]
    y_valid = y.iloc[test_index]

    lgbm_x_train = pd.DataFrame(train_df.iloc[train_index])
    lgbm_x_valid = pd.DataFrame(train_df.iloc[test_index])

    
    ########## LGBM model ##########
    lgbm_model = LGBMRegressor(
        random_state=random_state,
        learning_rate=0.005136975656960963,
        max_depth=9,
        num_leaves= 55,
        reg_alpha=0.20817676381652056,
        reg_lambda= 0.3934488653297847,
        n_estimators= 255,
        colsample_bytree= 0.898763145328619,
        min_child_samples= 32,
        subsample_freq = 4,
        subsample = 0.8626632385864561,
        max_bin= 170,
        min_data_per_group=32,
        cat_smooth=77,
        cat_l2= 20,
        verbose=-1,
    )
    lgbm_model.fit(
        train_df,
        y,
        eval_set=[(lgbm_x_valid, y_valid)], 
        verbose=0,
    )

    train_oof_preds = lgbm_model.predict(lgbm_x_valid)
    # test_oof_preds = lgbm_model.predict(test_df)
    lgbm_train_preds[test_index] = train_oof_preds
    # lgbm_test_preds += test_oof_preds / n_folds
    
    print(": LGB - RMSE = {}".format(mean_absolute_percentage_error(y_valid, train_oof_preds)))

--> Fold 1
: LGB - RMSE = 22.017476340807537
--> Fold 2
: LGB - RMSE = 73.33609996810625
--> Fold 3
: LGB - RMSE = 36.44202949811168
--> Fold 4
: LGB - RMSE = 205.78513993635454
--> Fold 5
: LGB - RMSE = 29.852019454832853
--> Fold 6
: LGB - RMSE = 27.09309628828924
--> Fold 7
: LGB - RMSE = 44.85831169299779
--> Fold 8
: LGB - RMSE = 42.697217663715165
--> Fold 9
: LGB - RMSE = 15.956568147038784
--> Fold 10
: LGB - RMSE = 14.181854661347812
--> Fold 11
: LGB - RMSE = 61.88464801418783
--> Fold 12
: LGB - RMSE = 214.94726130721472
--> Fold 13
: LGB - RMSE = 214.89305429940734
--> Fold 14
: LGB - RMSE = 54.965125386439865
--> Fold 15
: LGB - RMSE = 26.11421677649058
--> Fold 16
: LGB - RMSE = 24.895203114889668
--> Fold 17
: LGB - RMSE = 54.478542537794034
--> Fold 18
: LGB - RMSE = 19.179719371480573
--> Fold 19
: LGB - RMSE = 223.8723041230306
--> Fold 20
: LGB - RMSE = 58.14628863296273
--> Fold 21
: LGB - RMSE = 21.036006972262275
--> Fold 22
: LGB - RMSE = 33.54067699442892
--> Fo

In [73]:
test_oof_preds = lgbm_model.predict(test_df)

In [75]:
mean_absolute_percentage_error(test_quant_real,test_oof_preds)

4.22757382912277

---


# MLB IN only time series


In [298]:
mlb_in_train_times = mlb_in_train_1[['year', 'month', 'date']]
mlb_in_tests_times = mlb_in_tests_1[['year', 'month', 'date']]

In [299]:
lin_reg = LinearRegression()
modelfit(lin_reg, mlb_in_train_times, mlb_in_train_tar)


Model Report
RMSE :  58874.4550
CV Score : Mean -  49767.1451 | Std -  33095.8576 | Min -  22088.0818 | Max - 116729.1548
Amount of time spent during training the model and cross validation: 0.113 seconds


In [302]:
forest_reg = RandomForestRegressor(n_estimators=3000, 
                                   random_state=529, 
                                   min_samples_leaf=5,
                                   min_samples_split = 5,
                                   max_depth = 25,
                                   n_jobs=-1, oob_score=True)
modelfit(forest_reg, mlb_in_train_times, mlb_in_train_tar)


Model Report
RMSE :  52608.8477
CV Score : Mean -  50757.9233 | Std -  31504.5419 | Min -  24113.8041 | Max - 115324.7016
Amount of time spent during training the model and cross validation: 76.703 seconds


In [301]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

tree_reg = DecisionTreeRegressor(
    random_state=421, max_depth = 15, min_samples_split = 5)
modelfit(tree_reg, mlb_in_train_1, mlb_in_train_tar) 

# tree_ada = DecisionTreeRegressor(random_state = 42,max_depth = 4)

ada_reg = AdaBoostRegressor(
    tree_reg, n_estimators=300, random_state=42,learning_rate=0.009, loss='square')
modelfit(ada_reg, mlb_in_train_times, mlb_in_train_tar)

# 181.589


Model Report
RMSE :  19780.1639
CV Score : Mean -  88137.3955 | Std -  42987.0781 | Min -  26775.9041 | Max - 162303.5689
Amount of time spent during training the model and cross validation: 0.138 seconds

Model Report
RMSE :  9552.1449
CV Score : Mean -  50228.3440 | Std -  33095.4794 | Min -  20021.6018 | Max - 113961.1782
Amount of time spent during training the model and cross validation: 7.497 seconds


In [308]:
forest_reg_pre2 = pd.concat(
    [pd.DataFrame(forest_reg.predict(mlb_in_tests_times)), test_quant_real], 
    axis = 1,
    join = 'outer',
    ignore_index = True);
forest_reg_pre2

,0,1
0,9824.572655,7295.0
1,9933.735421,11790.0
2,11359.208885,27832.0
3,26523.414284,172755.0
4,30245.766592,7432.0
5,13893.093600,80613.0
6,25438.005251,10810.0
7,28192.603881,16623.0
8,28148.431259,58582.0
9,26132.864725,13519.0


In [368]:
metrics.mean_absolute_percentage_error(forest_reg_pre2[1],forest_reg_pre2[0])

3.1259660819336483

In [307]:

ada_reg_pre2 = pd.concat(
    [pd.DataFrame(ada_reg.predict(mlb_in_tests_times)), test_quant_real], 
    axis = 1,
    join = 'outer',
    ignore_index = True);
ada_reg_pre2

,0,1
0,13132.000000,7295.0
1,20429.333333,11790.0
2,345.500000,27832.0
3,56.250000,172755.0
4,48.666667,7432.0
5,1922.000000,80613.0
6,35381.500000,10810.0
7,42543.000000,16623.0
8,25601.666667,58582.0
9,11824.000000,13519.0


In [369]:
metrics.mean_absolute_percentage_error(ada_reg_pre2[1],ada_reg_pre2[0])

4.345039470809149

---

# outbOund MLB



In [311]:
mlb_out = pd.read_csv('./mlb_ou.csv')
mlb_out.dropna(subset=['SKU', '品名'], inplace=True)
(mlb_out).shape


In [321]:
mlb_out.columns = ['date','code','sku','name','quant','loc','province','city']

In [322]:
mlb_out

,date,code,sku,name,quant,loc,province,city
0,2021/2/4 15:11,74716.0,71HDM104150L120,8809727312654DAIMAROO 共用 COMFORTABLE 帽衫,1,A01-B-118,四川省,成都市
1,2021/2/4 15:11,74717.0,71SKM104150L120,8809727314573DAIMAROO 女士 REGULAR 裙子,1,A02-C-115,四川省,成都市
2,2021/2/4 15:02,75190.0,71MTF206143D155,8809727319110DAIMAROO 共用 COMFORTABLE 卫衣,1,A01-C-102,浙江省,杭州市
3,2021/2/4 15:08,75209.0,71TS2504150L120,8809727315730DAIMAROO 共用 COMFORTABLE 短袖领T恤,1,A02-C-019,广东省,广州市
4,2021/2/4 16:53,75246.0,72SHC101150I220,8809703531659饰品 共用,1,A07-D-016,四川省,成都市
...,...,...,...,...,...,...,...,...
467622,2021/6/19 23:28,NaN,3ASXXA11N07INS260,8809792297979BASIC 运动鞋,10,MT-03,江苏省,无锡市
467623,2021/6/19 23:28,NaN,3ASXXA11N07INS265,8809792279517BASIC 运动鞋,10,MT-03,江苏省,无锡市
467624,2021/6/19 23:28,NaN,3ASXXA11N07INS270,8809792297986BASIC 运动鞋,10,MT-03,江苏省,无锡市
467625,2021/6/19 23:28,NaN,3ASXXA11N07INS275,8809792279524BASIC 运动鞋,10,MT-03,江苏省,无锡市


In [320]:
def data_ppp(mlb_in):
    mlb_in['top'] = mlb_in['name'].str.extract(r'(衣|夹克|衫|服|马甲|恤)').isnull() != True
    mlb_in['top'] = mlb_in['top'].astype(int)
    mlb_in['sp'] = mlb_in['name'].str.extract(r'(饰品|配饰)').isnull() != True
    mlb_in['sp'] = mlb_in['sp'].astype(int)
    mlb_in['cap'] = mlb_in['name'].str.extract(r'(.+CAP|帽)').isnull() != True
    mlb_in['cap'] = mlb_in['cap'].astype(int)
    mlb_in['pack'] = mlb_in['name'].str.extract(r'(.+PACK|包)').isnull() != True
    mlb_in['pack'] = mlb_in['pack'].astype(int)
    mlb_in['female'] = mlb_in['name'].str.extract(r'(女士)').isnull() != True
    mlb_in['female'] = mlb_in['female'].astype(int)
    mlb_in['pants'] = mlb_in['name'].str.extract(r'(裤|PANT)').isnull() != True
    mlb_in['pants'] = mlb_in['pants'].astype(int)
    mlb_in['OVERSIZED'] = mlb_in['name'].str.extract(r'(.+OVERSIZED)').isnull() != True
    mlb_in['OVERSIZED'] = mlb_in['OVERSIZED'].astype(int)
 
    mlb_in['DAIMAROO'] = mlb_in['name'].str.extract(r'(DAIMAROO)').isnull() != True
    mlb_in['DAIMAROO'] = mlb_in['DAIMAROO'].astype(int)
    mlb_in['COMFORTABLE'] = mlb_in['name'].str.extract(r'(.+COMFORTABLE)').isnull() != True
    mlb_in['COMFORTABLE'] = mlb_in['COMFORTABLE'].astype(int)

    mlb_in['tmjl'] = mlb_in['name'].str.extract(r'(天猫精灵)').isnull() != True
    mlb_in['tmjl'] = mlb_in['tmjl'].astype(int)
    mlb_in['share'] = mlb_in['name'].str.extract(r'(共用)').isnull() != True
    mlb_in['share'] = mlb_in['share'].astype(int)
    mlb_in['shoe'] = mlb_in['name'].str.extract(r'(鞋)').isnull() != True
    mlb_in['shoe'] = mlb_in['shoe'].astype(int)
    mlb_in['cup'] = mlb_in['name'].str.extract(r'(杯)').isnull() != True
    mlb_in['cup'] = mlb_in['cup'].astype(int)
    mlb_in['textile'] = mlb_in['name'].str.extract(r'(纺织品)').isnull() != True
    mlb_in['textile'] = mlb_in['textile'].astype(int)

    mlb_in['socks'] = mlb_in['name'].str.extract(r'(袜)').isnull() != True
    mlb_in['socks'] = mlb_in['socks'].astype(int)

    mlb_in['dress'] = mlb_in['name'].str.extract(r'(裙)').isnull() != True
    mlb_in['dress'] = mlb_in['dress'].astype(int)
    mlb_in['suit'] = mlb_in['name'].str.extract(r'(套装)').isnull() != True
    mlb_in['suit'] = mlb_in['suit'].astype(int)
    mlb_in['fan'] = mlb_in['name'].str.extract(r'(风扇)').isnull() != True
    mlb_in['fan'] = mlb_in['fan'].astype(int)

data_ppp(mlb_out)


In [341]:
mlb_out['yearmon'] = mlb_out['date'].str.extract(r'(.{9})')
mlb_out_train = mlb_out.groupby(['yearmon'], as_index=False).sum()

In [353]:
mlb_out_train.head()

,yearmon,code,quant,top,sp,cap,pack,female,pants,OVERSIZED,...,COMFORTABLE,tmjl,share,shoe,cup,textile,socks,dress,suit,fan
0,2021/2/1,20659348.0,20065,871,51,524,143,7,40,2,...,56,0,114,444,0,13,40,3,0,0
1,2021/2/10,10561131.0,3567,651,17,31,51,1,39,4,...,35,0,59,102,0,4,0,1,0,0
2,2021/2/11,4900332.0,65,20,19,16,17,0,4,0,...,8,0,28,2,0,1,0,0,0,0
3,2021/2/12,5056360.0,66,39,16,17,1,0,6,0,...,15,0,32,0,0,0,0,1,0,0
4,2021/2/13,5366371.0,70,37,23,24,0,1,6,0,...,22,0,46,1,0,1,0,0,0,0
